In [1]:
!pip install bert-score --no-deps --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from nltk.metrics.distance import edit_distance
from nltk.translate.meteor_score import meteor_score
from nltk.corpus import wordnet as wn
from bert_score import score as bert_score

# helper for WordNet similarity
def best_wn_similarity(w1, w2):
    syns1 = wn.synsets(w1)
    syns2 = wn.synsets(w2)
    maxsim = 0.0
    for s1 in syns1:
        for s2 in syns2:
            sim = s1.wup_similarity(s2) or 0.0
            if sim > maxsim:
                maxsim = sim
    return maxsim

file_paths = [
    '/kaggle/input/pali-svqa-eval-finetuned/PaliGemma-Finetuned.csv',
    '/kaggle/input/blip-svqa-fine-eval/Blip-Finetuned.csv'
]

for path in file_paths:
    df = pd.read_csv(path).dropna(subset=['ground_truth','prediction'])
    GT = df['ground_truth'].str.lower().tolist()
    Y  = df['prediction'].str.lower().tolist()

    # Exact‐Match Accuracy
    acc = accuracy_score(GT, Y)

    # Macro F1
    prec, rec, f1_macro, _ = precision_recall_fscore_support(
        GT, Y, average='macro', zero_division=0
    )
    # WordNet Wu‐Palmer
    wn_sims = [best_wn_similarity(y, g) for y, g in zip(Y, GT)]
    avg_wn = np.mean(wn_sims)

    # BERTScore
    P, R, F1 = bert_score(
        cands=Y, refs=GT, lang="en", rescale_with_baseline=True
    )

    model_name = path.split('/')[-1]
    print(f"\n=== {model_name} ===")
    print(f"Exact‐Match Accuracy    : {acc:.3f}")
    print(f"Macro F1    : {f1_macro:.3f}")
    print(f"Wu–Palmer Sim      : {avg_wn:.3f}")
    print(f"BERT F1    : {F1.mean():.3f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== PaliGemma-Finetuned.csv ===
Exact‐Match Accuracy    : 0.739
Macro F1    : 0.452
Wu–Palmer Sim      : 0.904
BERT F1    : 0.940


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Blip-Finetuned.csv ===
Exact‐Match Accuracy    : 0.456
Macro F1    : 0.131
Wu–Palmer Sim      : 0.797
BERT F1    : 0.880
